# Wrangling Data with MongoDB

## Conecção com o banco de dados

In [1]:
!pip install pymongo --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 23.4 MB/s eta 0:00:00


* Importação das bibliotecas

In [2]:
from pprint import PrettyPrinter
import pandas as pd
from pymongo import MongoClient
from pymongo.server_api import ServerApi

* Conex

In [3]:
uri = ""
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


## Prepração dos dados

### Exploração

In [4]:
pp = PrettyPrinter(indent=2)

* Verificando quais dos bancos de dados estão disponíveis no cliente.

In [5]:
pp.pprint(list(client.list_databases()))

[ {'empty': False, 'name': 'air-quality', 'sizeOnDisk': 16142336},
  {'empty': False, 'name': 'sample_airbnb', 'sizeOnDisk': 60510208},
  {'empty': False, 'name': 'sample_analytics', 'sizeOnDisk': 15810560},
  {'empty': False, 'name': 'sample_geospatial', 'sizeOnDisk': 2125824},
  {'empty': False, 'name': 'sample_mflix', 'sizeOnDisk': 129335296},
  {'empty': False, 'name': 'sample_restaurants', 'sizeOnDisk': 10752000},
  {'empty': False, 'name': 'sample_supplies', 'sizeOnDisk': 2113536},
  {'empty': False, 'name': 'sample_weatherdata', 'sizeOnDisk': 4505600},
  {'empty': False, 'name': 'admin', 'sizeOnDisk': 368640},
  {'empty': False, 'name': 'local', 'sizeOnDisk': 24794124288}]


* Atribuindo o banco de dados "air-quality" à variável "db".
* Atribuindo a coleção "nairobi" à variável "nairobi"

In [6]:
db = client["air-quality"]
nairobi = db["nairobi"]

* Verificando quantos documentos existem na coleção "nairobi"

In [7]:
nairobi.count_documents({})

202212

* Usando o método find_one para recuperar um documento da coleção "nairobi" e atribuí-lo à variável nome result.

In [8]:
result = nairobi.find_one({})
pp.pprint(result)

{ '_id': '6525d772f44bfedd842a6fcc',
  'metadata': { 'lat': -1.3,
                'lon': 36.785,
                'measurement': 'temperature',
                'sensor_id': 58,
                'sensor_type': 'DHT22',
                'site': 29},
  'temperature': 16.5,
  'timestamp': '2018-09-01T00:00:04.301000'}


* Usendo o método dinstict para determinar quantos locais de sensores estão incluídos na coleção de Nairóbi.

In [9]:
nairobi.distinct("metadata.site")

[6, 29]

* Usando o método count_documents para determinar quantas leituras há para cada site na coleção de Nairobi.

In [10]:
print("Documents from site 6:", nairobi.count_documents({"metadata.site":6}))
print("Documents from site 29:", nairobi.count_documents({"metadata.site":29}))

Documents from site 6: 70360
Documents from site 29: 131852


* Usando o método aggregate para determinar quantas leituras há para cada local na coleção de Nairóbi.

In [11]:
result = nairobi.aggregate(
    [
        {"$group": {"_id": "$metadata.site", "count": {"$count": {}}}}
    ]
)
pp.pprint(list(result))

[{'_id': 6, 'count': 70360}, {'_id': 29, 'count': 131852}]


* Utilizando o método distinct para determinar quantos tipos de medições foram feitas na coleção de Nairóbi.

In [12]:
nairobi.distinct("metadata.measurement")

['P1', 'P2', 'humidity', 'temperature']

* Utilizando o método find para recuperar as leituras de PM 2,5 de todos os locais.

In [13]:
result = nairobi.find({"metadata.measurement": "P2"}).limit(3)
pp.pprint(list(result))

[ { 'P2': 34.43,
    '_id': '6525d775f44bfedd842bf24d',
    'metadata': { 'lat': -1.3,
                  'lon': 36.785,
                  'measurement': 'P2',
                  'sensor_id': 57,
                  'sensor_type': 'SDS011',
                  'site': 29},
    'timestamp': '2018-09-01T00:00:02.472000'},
  { 'P2': 30.53,
    '_id': '6525d775f44bfedd842bf24e',
    'metadata': { 'lat': -1.3,
                  'lon': 36.785,
                  'measurement': 'P2',
                  'sensor_id': 57,
                  'sensor_type': 'SDS011',
                  'site': 29},
    'timestamp': '2018-09-01T00:05:03.941000'},
  { 'P2': 22.8,
    '_id': '6525d775f44bfedd842bf24f',
    'metadata': { 'lat': -1.3,
                  'lon': 36.785,
                  'measurement': 'P2',
                  'sensor_id': 57,
                  'sensor_type': 'SDS011',
                  'site': 29},
    'timestamp': '2018-09-01T00:10:04.374000'}]


* Utilizando o método aggregate para calcular quantas leituras há para cada tipo ("umidade", "temperatura", "P2" e "P1") no local 6.

In [14]:
result = nairobi.aggregate(
    [
        {"$match": {"metadata.site": 6}},
        {"$group": {"_id": "$metadata.measurement", "count": {"$count": {}}}}
    ]
)
pp.pprint(list(result))

[ {'_id': 'P1', 'count': 18169},
  {'_id': 'humidity', 'count': 17011},
  {'_id': 'temperature', 'count': 17011},
  {'_id': 'P2', 'count': 18169}]


* Utilizando o método agregado para calcular quantas leituras há para cada tipo ("umidade", "temperatura", "P2" e "P1") no local 29.

In [15]:
result = nairobi.aggregate(
    [
        {"$match": {"metadata.site": 29}},
        {"$group": {"_id": "$metadata.measurement", "count": {"$count": {}}}}
    ]
)
pp.pprint(list(result))

[ {'_id': 'P1', 'count': 32907},
  {'_id': 'humidity', 'count': 33019},
  {'_id': 'P2', 'count': 32907},
  {'_id': 'temperature', 'count': 33019}]


## Importação

* Utilizando o método find para recuperar as leituras de PM 2,5 do local 29. Como não precisaremos dos metadados para o nosso modelo, vamos utilizar o argumento "projection" para limitar os resultados apenas às chaves "P2" e "timestamp".

In [16]:
result = nairobi.find(
    {"metadata.site": 29, "metadata.measurement": "P2"},
    projection={"P2": 1, "timestamp": 1, "_id": 0}
)
pp.pprint(result.next())

{'P2': 34.43, 'timestamp': '2018-09-01T00:00:02.472000'}


* Colocando os registros do result no DataFrame df.

In [17]:
df = pd.DataFrame(result).set_index("timestamp")
df.head()

,P2
timestamp,
2018-09-01T00:05:03.941000,30.53
2018-09-01T00:10:04.374000,22.80
2018-09-01T00:15:04.245000,13.30
2018-09-01T00:20:04.869000,16.57
2018-09-01T00:25:04.659000,14.07
